## Matrix Factorization
行列分解の手法

### 行列の次元削減
M×Nの行列Rを任意のkで、M×kとN×kの2つの行列U,Vに分解する  
$$
R=U \times V^T
$$

$R : 5 \times 8$の行列を
$k=2$で近似する例  

$$
\begin{pmatrix}
0 & 3 & 3 & 2 & 5 & 5 & 5 & 0 \\
1 & 1 & 5 & 3 & 2 & 1 & 1 & 5 \\
3 & 5 & 0 & 3 & 3 & 5 & 4 & 0 \\
2 & 1 & 2 & 1 & 0 & 1 & 1 & 4 \\
4 & 1 & 0 & 1 & 4 & 5 & 2 & 5 
\end{pmatrix}
\thickapprox
\begin{pmatrix}
2.185 & 0.066 \\
0.317 &  1.947 \\
2.248 & -0.139 \\
0.166 &  1.292 \\
1.284 &  1.295 
\end{pmatrix}
\times
\begin{pmatrix}
0.877 & 1.676 0.401 & 1.002 & 1.876 & 2.386 & 1.934 & 0.163\\
1.015 & 0.022  & 1.618 0.816 & 0.64 & 0.519 & 0.159  &2.90 
\end{pmatrix}
$$

kの個数を増やすことでもとの行列を近似精度を上げる。  
U,VはもとのRとの2乗誤差の勾配を用いて、更新することでU,Vを求める。  


誤差関数$E$を定義する。

$$
E = \sum_{i}^M \sum_j^N (R_{i,j}-\sum_{k=0}^k U_{i,k} \times V^T_{k,j})^2 +\lambda (\sum_{i,k} U_{i,k}^2+\sum_{k,j} V^2_{k,j})
$$

$U_{i,k}$の更新式は, $E$の勾配から  
$$
\Delta U_{i,k} = -\frac{\partial E}{\partial U_{i,k}} 
$$
$$
= (R_{i,j}-\sum_{k=0}^k U_{i,k} \times V^T_{k,j}) \times V^T_{k,j}
$$
$$
\Delta V_{k,j} = -\frac{\partial E}{\partial V_{k,j}}  
$$
$$
= (R_{i,j}-\sum_{k=0}^k U_{i,k} \times V^T_{k,j}) \times U_{i,k}
$$

numpyでの確認

In [304]:
import numpy as np

M=5
N=8
K=3

#R=np.arange(M*N).reshape((M, N))

#行列R
R=np.array([[0, 3, 3, 2, 5, 5, 5, 0],
            [1, 1, 5, 3, 2, 1, 1, 5],
            [3, 5, 0, 3, 3, 5, 4, 0],
            [2, 1, 2, 1, 0, 1, 1, 4],
            [4, 1, 0, 1, 4, 5, 2, 5]])

#行列U, V
U=np.random.rand(M,K)
V=np.random.rand(N,K)

#積計算
#r=np.dot(U, V.T)
#print(r)

#近似したR
r=np.zeros((M,N))

#誤差関数
E=np.zeros((M,N))

#繰り返し回数
for a in range(300):
    
    #近似したR
    r=np.dot(U, V.T)
    #誤差計算
    E=R-r

    #勾配計算
    for m in range(M):
        for n in range(N):
            for k in range(K):
                U[m,k]+=0.01 * (E[m,n]*V[n,k] -0.01*U[m,k])
                V[n,k]+=0.01 * (E[m,n]*U[m,k] -0.01*V[n,k])

    #誤差表示
    #print(np.sum(np.power(R-np.dot(U, V.T),2)))
#print(np.round(1000*U)/1000)
#print(np.round(1000*V.T)/1000)
#print(U)
#print(V.T)

#積計算
r=np.dot(U, V.T)
print('R')
print(R)
print('近似R')
print(np.round(r))

R
[[0 3 3 2 5 5 5 0]
 [1 1 5 3 2 1 1 5]
 [3 5 0 3 3 5 4 0]
 [2 1 2 1 0 1 1 4]
 [4 1 0 1 4 5 2 5]]
近似R
[[ 0.  4.  3.  3.  4.  5.  5. -0.]
 [ 1.  1.  5.  2.  2.  1.  1.  5.]
 [ 2.  4.  0.  2.  4.  6.  4.  0.]
 [ 2.  0.  2.  1.  1.  1.  0.  4.]
 [ 4.  2.  0.  2.  3.  5.  2.  5.]]


### 量子アニーリングでMF問題を解く
量子アニーリングでMF問題を解くときには、0と1の変数を使用する必要がある。
Uは実数で行列Vを0,1で構成する。

In [41]:
import numpy as np

np.random.seed(6)

M=5
N=8
K=3

#量子ビット
#q = Array.create('q', shape=(K), vartype='BINARY')

#R=np.arange(M*N).reshape((M, N))

#行列R
R=np.array([[0, 3, 3, 2, 5, 5, 5, 0],
            [1, 1, 5, 3, 2, 1, 1, 5],
            [3, 5, 0, 3, 3, 5, 4, 0],
            [2, 1, 2, 1, 0, 1, 1, 4],
            [4, 1, 0, 1, 4, 5, 2, 5]])

#行列U, V
U=np.random.rand(M,K)
V=np.ones((N,K))

#積計算
#r=np.dot(U, V.T)
#print(r)

#近似したR
r=np.zeros((M,N))

#誤差関数
E=np.zeros((M,N))

#繰り返し回数
for loop1 in range(10):
    
    #近似
    r=np.dot(U, V.T)
    #誤差計算
    E=R-r
    
    
    #U更新
    for m in range(M):
        for n in range(N):
            for loop2 in range(100):
                H=0;
                r=np.dot(U, V.T)
                E=R-r
                for k in range(K):
                    U[m,k]+=0.01 * (E[m,n]*V[n,k] -0.001*U[m,k])
                    
                for k in range(K):
                    H += U[m,k]*q[k] #Uのk列目との内積
                    
                #R[m,n]のハミルトニアン
                H=(R[m,n]-H)**2
            
                #ここにTYTAN記述
            
                #Uのビット取り出し
                for key, value in best_sample.sample.items():
                    key=key.replace('q[', '')
                    key=key.replace(']', '')
                    V[n,int(key)]=value
                    
                #収束判定
                r= np.dot(U, V.T)
                error = (R[m,n]-r[m,n])**2
                if error <=0.01:
                    #print(U)
                    #print(error)
                    break
                

    #誤差表示
    #print(np.sum(np.power(R-np.dot(U, V.T),2)))
#r=np.dot(U, V.T)  
#print(U)
#print(V.T)
print('U行列')
print(np.round(1000*U)/1000)
print('V.T行列')
print(V.T)
print('誤差')
print(np.sum(np.power(R-np.dot(U, V.T),2)))
#積計算
r=np.dot(U, V.T)
print('R')
print(R)
print('近似R')
#print(r)
print(np.round(r))
#print(np.round(100*r)/100)

U行列
[[1.917 1.078 1.795]
 [1.123 1.762 2.015]
 [2.13  1.668 1.173]
 [1.199 0.947 1.754]
 [1.118 1.768 2.1  ]]
V.T行列
[[0. 1. 0. 1. 0. 1. 0. 1.]
 [1. 0. 0. 0. 1. 1. 0. 1.]
 [1. 0. 0. 0. 1. 1. 1. 1.]]
誤差
174.54944752178437
R
[[0 3 3 2 5 5 5 0]
 [1 1 5 3 2 1 1 5]
 [3 5 0 3 3 5 4 0]
 [2 1 2 1 0 1 1 4]
 [4 1 0 1 4 5 2 5]]
近似R
[[3. 2. 0. 2. 3. 5. 2. 5.]
 [4. 1. 0. 1. 4. 5. 2. 5.]
 [3. 2. 0. 2. 3. 5. 1. 5.]
 [3. 1. 0. 1. 3. 4. 2. 4.]
 [4. 1. 0. 1. 4. 5. 2. 5.]]
